# ZeroShot (Algo 1)

    for total_batches in range(total_n_pseudo_batches):
    
        z = tf.random.normal([batch_size, z_dim])
        pseudo_images = get_gen_images(z)
        teacher_logits, *teacher_activations = get_model_outputs(teacher_model, pseudo_images, mode=0)

        #generator training
        for ng in range(ng_batches):
            student_logits, *student_activations = get_model_outputs(student_model, pseudo_images, mode=1)
            generator_loss = generator_loss(teacher_logits, student_logits)

            #################################
            # BACK PROP AND tick schedulers #
            #################################  

        for ns in range(ns_batches):
            student_logits, *student_activations = get_model_outputs(student_model, pseudo_images, mode=1)
            student_loss = student_loss(teacher_logits, teacher_activations, 
                                        student_logits, student_activations, attn_beta)

            #################################
            # BACK PROP AND tick schedulers #
            #################################   

        ######################################################
        ### Val accuracy computation and best model saving ###
        ######################################################    

In [1]:
import tensorflow as tf
from net.generator import NavieGenerator
from utils.cosine_anealing import CosineAnnealingScheduler
from utils.losses import student_loss_fn, generator_loss_fn
#from utils.losses import student_loss_fn
from tensorflow.keras.optimizers import Adam
from net.wide_resnet import WideResidualNetwork
from tensorflow.keras.experimental import CosineDecay
import numpy as np

In [2]:
tf.__version__

'2.0.0'

In [7]:
z_dim = 100
batch_size = 128
ng_batches = 1
ns_batches = 10
attn_beta = 250
total_n_pseudo_batches = 80000
student_lr = 2e-3
generator_lr = 1e-3

teacher = WideResidualNetwork(40, 2, input_shape=(32, 32, 3), dropout_rate=0.0, has_softmax=False, output_activations=True)
teacher.load_weights('cifar10_WRN-40-2-seed23_model.186.h5')
teacher.trainable = False

student = WideResidualNetwork(16, 1, input_shape=(32, 32, 3), dropout_rate=0.0, has_softmax=False, output_activations=True)
student_optimizer = Adam(learning_rate=CosineDecay(student_lr, total_n_pseudo_batches*ns_batches))

generator = NavieGenerator(input_dim=100)
generator_optimizer = Adam(learning_rate=CosineDecay(generator_lr, total_n_pseudo_batches*ng_batches))

# Generator loss metrics
g_loss_met = tf.keras.metrics.Mean()
# Student loss metrics
stu_loss_met = tf.keras.metrics.Mean()

In [8]:
for total_batches in range(total_n_pseudo_batches):
    # sample from latern space to make an image
    z = tf.random.normal([batch_size, z_dim])

    # Generator training
    generator.trainable = True
    student.trainable = False
    for ng in range(ng_batches):
        with tf.GradientTape() as tape:
            pseudo_imgs = generator(z)
            t_logits, *_ = teacher(pseudo_imgs)
            s_logits, *_ = student(pseudo_imgs)

            # calculate the generator loss
            loss = generator_loss_fn(t_logits, s_logits)

        # The grad for generator
        grads = tape.gradient(loss, generator.trainable_weights)

        # update the generator paramter with the gradient
        generator_optimizer.apply_gradients(zip(grads, generator.trainable_weights))

        g_loss_met(loss)

    print('step %s: generator mean loss = %s' % (total_batches, g_loss_met.result().numpy()))
    # ==========================================================================

    # Student training
    generator.trainable = False
    student.trainable = True
    for ns in range(ns_batches):

        #t_logits, *t_acts = teacher(pseudo_imgs)
        with tf.GradientTape() as tape:
            s_logits, *s_acts = student(pseudo_imgs)
            loss = student_loss_fn(t_logits, t_acts, s_logits, s_acts, attn_beta)

        # The grad for student
        grads = tape.gradient(loss, student.trainable_weights)

        # Apply grad for student
        student_optimizer.apply_gradients(zip(grads, student.trainable_weights))

        stu_loss_met(loss)

    print('step %s-%s: studnt mean loss = %s' % (total_batches, ns, stu_loss_met.result().numpy()))


step 0: generator mean loss = -2.246053
step 0-9: studnt mean loss = 0.8498529
step 1: generator mean loss = -1.1230265
step 1-9: studnt mean loss = 0.4274651


KeyboardInterrupt: 

In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [6]:
from utils.preprocess import load_cifar10_data

In [7]:
(x_train, y_train), (x_test, y_test) = load_cifar10_data()

In [36]:
def get_accuracy(student):
    model = WideResidualNetwork(16, 1, input_shape=(32, 32, 3), dropout_rate=0.0)
    model.set_weights(student.get_weights()) 
    model.compile(loss='categorical_crossentropy',
                      metrics=['accuracy'])
    loss, accuracy = model.evaluate(x_test, y_test, batch_size=128, verbose=0)
    return loss, accuracy

In [29]:
stest = student(x_test)

TypeError: call() got an unexpected keyword argument 'batch_size'

In [26]:
import timeit

In [45]:
%%time
get_accuracy(student)

Wall time: 8.73 s


(29.924496099853517, 0.1)

In [44]:
student.trainable = False

In [13]:
a = True
b = False

In [17]:
if a and not b:
    print("yes")

yes


In [33]:
%%time
model = WideResidualNetwork(16, 1, input_shape=(32, 32, 3), dropout_rate=0.0)
model.set_weights(student.get_weights()) 
model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

Wall time: 577 ms


In [31]:
%%time
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Test loss: 2.3115152214050294
Test accuracy: 0.1098
Wall time: 7.44 s


In [39]:
0%11

0

In [40]:
10%11

10

In [41]:
11%11

0

In [42]:
12%11

1

In [43]:
1%11

1

In [49]:
ip = 0
npb = 0
tnpb = 2
nrb = 11
ng = 1
ns = 10
while npb < tnpb:
    if ip % nrb < ng:
        print("gen update")
    elif ip % nrb < (ng+ns):
        print("student update")
    if (ip+1) % nrb == 0:
        npb += 1
    ip += 1

gen update
student update
student update
student update
student update
student update
student update
student update
student update
student update
student update
gen update
student update
student update
student update
student update
student update
student update
student update
student update
student update
student update


In [50]:
8e4

80000.0

In [74]:
student_optimizer.learning_rate(80001).numpy()

0.0

In [ ]:
CosineDecay()